In [1]:
"""DE Test Case."""
from pathlib import Path
import pandas as pd
import pomato

In [2]:
# wdir = Path("./examples/") # Change to local copy of examples folder
wdir = Path.cwd()
mato = pomato.POMATO(wdir=wdir, options_file="profiles/de.json")
mato.load_data('data_input/DE_2020.zip')


INFO - Creating Folder Structure
INFO - creating folder data_temp
INFO - creating folder data_temp/julia_files
INFO - creating folder data_temp/julia_files/data
INFO - creating folder data_temp/julia_files/results
INFO - creating folder data_temp/julia_files/cbco_data
INFO - Initializing DataObject
INFO - Initializing the GridTopology....
INFO - Initializing MarketModel...
INFO - Initializing the FBMCModule....
INFO - Initializing the GridTopology....
INFO - POMATO initialized!
INFO - Loading data from zipped data archive
WARNING - Not complete list of expected input data found. See .missing_data
INFO - Validating Input Data...
WARNING - Data validation completed with warnings. See the .data_validation_report.
WARNING - Some data was initialized empty. See model_validation_report.
WARNING - Some data missing or contained NaNs. See model_validation_report.
INFO - Calculating grid parameters!
INFO - The network consists of 25 components. Setting slack for each.
INFO - Calculating PTDF an

In [ ]:
# Create the grid representation, the options are specified in the 
# json file supplied in the instantiation. 
# In this case the model runs a zonal pricing with NTC to neighboring countries
# and subsequent redispatch.
mato.create_grid_representation()
mato.run_market_model(update_data=True)

rket Model for timestep t0001...
INFO - Academic license - for non-commercial use only
INFO - Adding Variables and Expressions..
INFO - Add Base Model
INFO - Adding Curtailment...
INFO - Adding NTC Constraints...
INFO - Adding Objective Function...
INFO - Solving...
INFO - Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
INFO - Optimize a model with 796717 rows, 1021608 columns and 2703792 nonzeros
INFO - Model fingerprint: 0x0b0c7cb3
INFO - Coefficient statistics:
INFO - Matrix range     [8e-01, 1e+00]
INFO - Objective range  [1e+00, 6e+02]
INFO - Bounds range     [1e+04, 1e+04]
INFO - RHS range        [2e-10, 2e+06]
INFO - 
INFO - Concurrent LP optimizer: primal simplex, dual simplex, and barrier
INFO - Showing barrier log only...
INFO - 
INFO - Presolve removed 708646 rows and 341935 columns (presolve time = 5s) ...
INFO - Presolve removed 716720 rows and 341935 columns
INFO - Presolve time: 5.79s
INFO - Presolved: 79997 rows, 687727 columns, 1457932 nonzeros
INFO - 
INFO - Or

In [ ]:
# Return the results, in this case two: 
# market result (uniform pricing, no network constraints)
# and redispatch (to account for N-0 network feasibility)
market_result, redisp_result = mato.data.return_results()


In [ ]:
# Check for overloaded lines in the market and redispatch results
n0_m, _ = market_result.overloaded_lines_n_0()
print("Number of N-0 Overloads in market result: ", len(n0_m))
n0_r, _  = redisp_result.overloaded_lines_n_0()
print("Number of N-0 Overloads after redispatch: ", len(n0_r))

Number of N-0 Overloads in market result:  259
Number of N-0 Overloads after redispatch:  25


In [ ]:
# Generation comparison between Market Result and Redispatch.
# Redispatch is calculated G_redispatch - G_market as delta
# The absolute delta represents the total redispatched energy
gen = pd.merge(market_result.data.plants[["plant_type", "fuel", "technology", "g_max", "node"]],
               market_result.G, left_index=True, right_on="p")
gen = pd.merge(gen, redisp_result.G, on=["p", "t"], suffixes=("_market", "_redispatch"))
gen["delta"] = gen["G_redispatch"] - gen["G_market"]
gen["delta_abs"] = gen["delta"].abs()
print("Redispatched energy per hour (abs) [MWh] ", gen.delta_abs.sum()/len(gen.t.unique()))

# Many of these functions are implemented as methods of the Result class
gen_2 = redisp_result.redispatch()
print("Redispatched energy per hour (abs) [MWh] ", gen_2.delta_abs.sum()/len(gen_2.t.unique()))


Redispatched energy per hour (abs) [MWh]  1125.0263326531328
Redispatched energy per hour (abs) [MWh]  1125.0263326531328


In [ ]:
# Create Geo Plot. DISCLAiMER: The reported prices are the dual 
# in the redispatch result, thus including costs for redispatch.
mato.visualization.create_geo_plot(redisp_result, show_prices=True, show_plot=False, show_redispatch=True, filepath=mato.wdir.joinpath("redispatch.html"))


INFO - Precalculating and caching common results..
INFO - Estimated size in RAM for A is: 21.476448 MB
INFO - Done Calculating N-1 Flows
INFO - Done calculating common results.


In [ ]:
# Create visualization of the generation schedule in the market result. 
mato.visualization.create_generation_plot(market_result, show_plot=False, filepath=mato.wdir.joinpath("market_results.html"))



In [ ]:
# 
mato._join_julia_instances()


INFO - EXIT
